In [1]:
from hypergraphs import *

In [2]:
carbohydrateMetabolismPathways = ["00010","00020","00030","00040","00051","00052","00053","00500","00520","00620","00630","00640","00650","00660","00562"]

In [3]:
H, sr2r, r2ss, rid2rname, mid2mname = read_hg_from_SBML(["../../LaTeX/Data/hsa/hsa{}-sbml.xml".format(x) for x in carbohydrateMetabolismPathways])

/Users/boldi/.local/share/virtualenvs/qf-BIT_cIX4/lib/python3.8/site-packages/hypernetx/classes/hypergraph.py:968: UserWarning: Cannot add edge. Edge already in hypergraph
  warnings.warn("Cannot add edge. Edge already in hypergraph")


Ignoring file ../../LaTeX/Data/hsa/hsa00660-sbml.xml, couldn't get a model out of it


In [4]:
dd=hyper_cardon_crochemore(H)

In [5]:
showMapKeys(dd, onlyNonTrivial=True, keyadj=sr2r, prefOnly="r", keyfamily=r2ss)

{'rnR01529', 'rnR01639'}
{'rnR00238', 'rnR00410', 'rnR01975'}
{'rnR01819', 'rnR02740', 'rnR00768', 'rnR04779', 'rnR03321'}
{'rnR01057', 'rnR01051', 'rnR01049', 'rnR01056'}
{'rnR03469', 'rnR05795', 'rnR11680'}
{'rnR09827', 'rnR04513'}
{'rnR03394', 'rnR05800', 'rnR03433'}
{'rnR00200', 'rnR00658'}
{'rnR01518', 'rnR01512', 'rnR01516'}
{'rnR01104', 'rnR01329'}
{'rnR01518', 'rnR00658'}
{'rnR01057', 'rnR01051', 'rnR01049'}
{'rnR08639', 'rnR00289', 'rnR00959', 'rnR01660'}
{'rnR00955', 'rnR00503', 'rnR00291'}
{'rnR01104', 'rnR01329'}
{'rnR02750', 'rnR02749'}
{'rnR00746', 'rnR00754'}
{'rnR00867', 'rnR00866', 'rnR00760', 'rnR00875'}
{'rnR00945', 'rnR01394', 'rnR01221', 'rnR01392'}
{'rnR01896', 'rnR01639'}
{'rnR07324', 'rnR08639', 'rnR00303', 'rnR00771'}
{'rnR01526', 'rnR01529', 'rnR01056'}
{'rnR00405', 'rnR00432', 'rnR00833'}
{'rnR01804', 'rnR02705', 'rnR01207'}
{'rnR01805', 'rnR04435'}
{'rnR02640', 'rnR02933'}
{'rnR04780', 'rnR01070'}
{'rnR00710', 'rnR00711', 'rnR00316', 'rnR00235'}
{'rnR08193',

In [6]:
showMapKeys(dd, onlyNonTrivial=True, keydesc=mid2mname, prefOnly="C")

{'C04063 (C04063)', 'C04477 (C04477)'}
{'C11555 (C11555)', 'C01220 (C01220)'}
{'C04520 (C04520)', 'C01204 (C01204)'}
{'C00026 (C00026)', 'C05379 (C05379)'}
{'C00190 (C00190)', 'C03033 (C03033)'}
{'C00641 (C00641)', 'C05981 (C05981)'}
{'C00119 (C00119)', 'C00620 (C00620)', 'C00121 (C00121)'}
{'C00168 (C00168)', 'C00037 (C00037)'}
{'C00258 (C00258)', 'C00014 (C00014)', 'C01146 (C01146)', 'C00065 (C00065)'}
{'C00672 (C00672)', 'C01801 (C01801)'}
{'C00181 (C00181)', 'C00312 (C00312)'}
{'C18060 (C18060)', 'C00334 (C00334)'}
{'C00232 (C00232)', 'C00203 (C00203)'}
{'C01177 (C01177)', 'C00492 (C00492)', 'C01674 (C01674)', 'C00058 (C00058)', 'C00185 (C00185)', 'C00007 (C00007)', 'C03827 (C03827)', 'C00999 (C00999)', 'C00064 (C00064)', 'C00136 (C00136)', 'C00257 (C00257)', 'C21384 (C21384)', 'C05402 (C05402)', 'C20238 (C20238)'}
{'C01720 (C01720)', 'C00252 (C00252)', 'C05404 (C05404)', 'C06002 (C06002)', 'C20934 (C20934)', 'C00198 (C00198)', 'C00025 (C00025)', 'C02336 (C02336)', 'C01083 (C01083)

In [7]:
classSizeDist(dd, prefOnly="r", keyadj=sr2r)

{1: 66, 2: 44, 3: 28, 5: 5, 4: 14, 7: 1, 28: 1}

In [8]:
classSizeDist(dd, prefOnly="C")

{1: 126, 2: 15, 3: 1, 4: 1, 14: 1, 26: 1, 10: 1}

In [47]:
import math
def writeKEGGHtml(m, dirname, basename, title, onlyNonTrivial = True, keyadj = None, 
                  prefOnly = "", cutPref="", addHtmlPrefix="https://www.kegg.jp/entry/", maxPerRow = 3):
    """
    """
    pageCounter = 0
    npages = len(set(m.values()))
    for v in set(m.values()):
        if keyadj is not None:
            eqClass = set([keyadj[k] for k,w in m.items() if w==v and k.startswith(prefOnly)])
        else:
            eqClass = set([k for k,w in m.items() if w==v and k.startswith(prefOnly)])
        if onlyNonTrivial and len(eqClass) <= 1:
            continue
        out = set(["{}".format(k) for k in eqClass])
        outn = len(out)
        rows = math.ceil(outn/maxPerRow)
        fileName = "{}-cont-{}.html".format(basename, pageCounter)
        prevName = "{}-cont-{}.html".format(basename, pageCounter - 1)
        nextName = "{}-cont-{}.html".format(basename, pageCounter + 1)
        headerName = "{}-header-{}.html".format(basename, pageCounter)
        footerName = "{}-footer-{}.html".format(basename, pageCounter)
        pageCounter += 1
        with open("{}/{}".format(dirname, fileName), "w") as file:
            file.write("<!DOCTYPE html>\n<html>\n\t<frameset rows=\"10%," +
                ",".join(["*"] * rows) +
                ",10%\">\n")
            file.write("\n\t\t<frame src=\"" + headerName + "\">\n")
            for i,element in enumerate(out):
                if i % maxPerRow == 0:
                    file.write("\t\t<frameset cols=\"" +
                        ",".join(["*"] * maxPerRow) +
                        "\">\n")
                file.write("\t\t\t<frame src=\"{}{}\">\n".format(addHtmlPrefix, element[len(cutPref):]))
                if (i + 1) % maxPerRow == 0 or i == outn - 1:
                    file.write("\t\t</frameset>\n")
            file.write("\n\t\t<frame src=\"" + footerName + "\">\n")
            file.write("\n\t</frameset>\n</html>")
        with open("{}/{}".format(dirname, headerName), "w") as file:
            file.write("<!DOCTYPE html>\n<html>\n\t<h1>{}{}</h1>\n</html>".format(title, pageCounter))
        with open("{}/{}".format(dirname, footerName), "w") as file:
            if pageCounter == 1:
                file.write("<!DOCTYPE html>\n<html>\n" +
                       "\t<table border=\"1\" style=\"margin-left: auto; margin-right: auto;\">\n" +
                       "\t\t<td style=\"padding: 10px;\"><a href=\"" + nextName + "\" target=\"_top\">Next</a></td>\n" +
                       "\t</table>\n" +
                       "</html>")
            elif pageCounter == npages:
                file.write("<!DOCTYPE html>\n<html>\n" +
                       "\t<table border=\"1\" style=\"margin-left: auto; margin-right: auto;\">\n" +
                       "\t\t<td style=\"padding: 10px;\"><a href=\"" + prevName + "\" target=\"_top\">Previous</a></td>\n" +
                       "\t</table>\n" +
                       "</html>")
            else:
                file.write("<!DOCTYPE html>\n<html>\n" +
                       "\t<table border=\"1\" style=\"margin-left: auto; margin-right: auto;\">\n" +
                       "\t\t<td style=\"padding: 10px;\"><a href=\"" + prevName + "\" target=\"_top\">Previous</a></td>\n" +
                       "\t\t<td style=\"padding: 10px;\"><a href=\"" + nextName + "\" target=\"_top\">Next</a></td>\n" +
                       "\t</table>\n" +
                       "</html>")


In [48]:
writeKEGGHtml(dd, "hsa", "carboR", "Homo Sapiens, Carbohydrate metabolism, Reactions group ", onlyNonTrivial=True, keyadj=sr2r, prefOnly="r", cutPref="rn")

In [49]:
writeKEGGHtml(dd, "hsa", "carboC", "Homo Sapiens, Carbohydrate metabolism, Compound group ", onlyNonTrivial=True, prefOnly="C", cutPref="")